# Instructions
You can use this notebook for **Task 1**

You can import the libraries that you need.
For instance:
```python
from sklearn.metrics import mean_squared_error
```

Please load the datasets that will be provided:
- Right-click and select upload from the left panel
- Select the files (*Japan_cities_rainfall.csv*, *rainfall_to_predict.csv*)


Below is the function to verify your final predictions based on the *rainfall_to_predict.csv* file.

**Please run it before starting**

#### Function to verify your prediction score
##### Root Mean Square Error

In order to verify the accuracy of you model, use the _verify_predictions_ function with your predictions results
```python
verify_predictions(output_result)

# returns {"result":RMSE_RESULT_VALUE,"type":"rmse"}
```


In [ ]:
# Please do not modify
import requests
import json

def verify_predictions(y_pred):
    
    url = "https://q7wmplg8u6.execute-api.ap-northeast-1.amazonaws.com/dev"
    
    payload = {
        "submission": y_pred
    }
    response = requests.request("POST", url,data=json.dumps(payload),headers = {'content-type': 'application/json'})
    
    return response.text

#Start your task from below

# Code for the Rainfall prediction



---



In [ ]:
# imports
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading the datasets
df = pd.read_csv('/content/drive/MyDrive/wni/Japan_cities_rainfall.csv')
df_to_predict = pd.read_csv('/content/drive/MyDrive/wni/rainfall_to_predict.csv')

In [ ]:
df.head()

,date_time,place,longitude,latitude,rainfall
0,20200918_080000,Naha,127.686667,26.206667,0.0
1,20200918_080000,Fukuoka,130.375000,33.581667,0.0
2,20200918_080000,Sendai,140.896667,38.261667,0.0
3,20200918_080000,Osaka,135.518333,34.681667,0.0
4,20200918_080000,Nigata,139.018333,37.893333,3.0


In [ ]:
df_to_predict.head()

,date_time,place,longitude,latitude
0,20200919_013000,Naha,127.686667,26.206667
1,20200919_223000,Nigata,139.018333,37.893333
2,20200919_233000,Sendai,140.896667,38.261667
3,20200920_030000,Fukuoka,130.375000,33.581667
4,20200920_130000,Naha,127.686667,26.206667


In [ ]:
def breakdown_month(a):
  if(a[4:6]=='9'):
      return 0
  elif(a[4:6]=='10'):
      return 1
  else:
      return 2
def breakdown_date(a):
  return int(a[6:8])
def breakdown_time(a):
  if(int(a[9:11])>=0 and int(a[9:11])<6):
    return 1
  elif(int(a[9:11])>=6 and int(a[9:11])<12):
    return 2
  elif(int(a[9:11])>=12 and int(a[9:11])<18):
    return 3
  elif(int(a[9:11])>=18 and int(a[9:11])<24):
    return 4
  

In [ ]:
city_mapping = {" Naha": 1, " Fukuoka": 2, " Sendai": 3, " Osaka": 4, " Nigata": 5, " Tokyo": 6}
datasets=[df,df_to_predict]
cols=['date','month','time','city','rainfall']
for data in datasets:
  data['city']=data['place'].map(city_mapping)
  data['month']=data['date_time'].apply(breakdown_month)
  data['date']=data['date_time'].apply(breakdown_date)
  data['time']=data['date_time'].apply(breakdown_time)
  data.drop(['place','longitude','latitude','date_time'],axis=1,inplace=True)
df=df[cols]
df_to_predict=df_to_predict[cols[:-1]]

In [ ]:
df.head()

,date,month,time,city,rainfall
0,18,2,2,1,0.0
1,18,2,2,2,0.0
2,18,2,2,3,0.0
3,18,2,2,4,0.0
4,18,2,2,5,3.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor  
Y=df['rainfall']
X=df.drop('rainfall',axis=1)
gbr_model=GradientBoostingRegressor()
gbr_model.fit(X, np.ravel(Y))
pred=gbr_model.predict(df_to_predict)
pred=pred.tolist()
result_rmse = verify_predictions(pred)
print(result_rmse)
print(df_to_predict[:10])
print(pred[:10])


{"result":0.7850236515182961,"type":"rmse"}

   date  month  time  city
0    19      2     1     1
1    19      2     4     5
2    19      2     4     3
3    20      2     1     2
4    20      2     3     1
5    20      2     4     1
6    20      2     4     5
7    21      2     2     5
8    21      2     3     3
9    21      2     4     5
[0.22822961314619544, 0.10497764087733628, 0.02135037157824171, 0.057531040927300776, 0.13906219945823972, 0.13361914782113057, 0.10497764087733628, 0.04139197265440486, 0.06135497316948109, 0.06319004782721582]


In [2]:
from flask import Flask,jsonify
import requests, json
import pickle

In [7]:
data=[
{'date':'2020-02-19','place':'naha','time':'060000','pred':0.22822961314619528},
{'date':'2020-02-19','place':'naha','time':'170000','pred':0.139062199458239},
{'date':'2020-02-19','place':'naha','time':'200000','pred':0.13361914782113052},
{'date':'2020-11-19','place':'sendai','time':'183000','pred':0.02135037157824171},
{'date':'2020-11-19','place':'nigata','time':'230000','pred':0.10497764087733633}]

In [11]:
app=Flask(__name__)
if __name__=="__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
@app.route('/data',methods=['GET'])
def get():
    return jsonify({"Data":data})

In [ ]:
pickle.dump(gbr_model,open('gbr_model.pkl','wb'))

In [ ]:
my_gbr=pickle.load(open('gbr_model.pkl','rb'))

In [ ]:
my_gbr

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
url="http://127.0.0.1:5000/"
data=json.dumps({'date':19,'month':0,'time':1,'city':1})
r=requests.post(url,data)
print(r.json())

ConnectionError: ignored